# Flux de lucru cu implicarea umană folosind Microsoft Agent Framework

## 🎯 Obiective de învățare

În acest notebook, vei învăța cum să implementezi fluxuri de lucru **cu implicarea umană** utilizând `RequestInfoExecutor` din Microsoft Agent Framework. Acest model puternic permite întreruperea fluxurilor de lucru AI pentru a colecta input uman, făcând agenții interactivi și oferind oamenilor control asupra deciziilor critice.

## 🔄 Ce este implicarea umană?

**Implicarea umană (HITL)** este un model de design în care agenții AI întrerup execuția pentru a solicita input uman înainte de a continua. Acest lucru este esențial pentru:

- ✅ **Decizii critice** - Obține aprobarea umană înainte de a lua măsuri importante
- ✅ **Situații ambigue** - Permite oamenilor să clarifice atunci când AI este nesigur
- ✅ **Preferințe ale utilizatorului** - Solicită utilizatorilor să aleagă între mai multe opțiuni
- ✅ **Conformitate și siguranță** - Asigură supravegherea umană pentru operațiuni reglementate
- ✅ **Experiențe interactive** - Construiește agenți conversaționali care răspund la inputul utilizatorului

## 🏗️ Cum funcționează în Microsoft Agent Framework

Framework-ul oferă trei componente cheie pentru HITL:

1. **`RequestInfoExecutor`** - Un executor special care întrerupe fluxul de lucru și emite un `RequestInfoEvent`
2. **`RequestInfoMessage`** - Clasa de bază pentru payload-uri de cereri tipizate trimise oamenilor
3. **`RequestResponse`** - Corelează răspunsurile umane cu cererile originale folosind `request_id`

**Modelul fluxului de lucru:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Exemplul nostru: Rezervare hotel cu confirmare utilizator

Vom construi pe baza fluxului de lucru condițional, adăugând confirmarea umană **înainte** de a sugera destinații alternative:

1. Utilizatorul solicită o destinație (de exemplu, "Paris")
2. `availability_agent` verifică dacă sunt camere disponibile
3. **Dacă nu sunt camere** → `confirmation_agent` întreabă "Doriți să vedeți alternative?"
4. Fluxul de lucru **se întrerupe** folosind `RequestInfoExecutor`
5. **Omul răspunde** "da" sau "nu" prin input în consolă
6. `decision_manager` direcționează în funcție de răspuns:
   - **Da** → Afișează destinații alternative
   - **Nu** → Anulează cererea de rezervare
7. Afișează rezultatul final

Acest exemplu demonstrează cum să oferi utilizatorilor control asupra sugestiilor agentului!

---

Să începem! 🚀


## Pasul 1: Importarea bibliotecilor necesare

Importăm componentele standard ale Agent Framework, plus **clasele specifice interacțiunii umane**:
- `RequestInfoExecutor` - Executor care oprește fluxul de lucru pentru a primi input uman
- `RequestInfoEvent` - Eveniment emis atunci când se solicită input uman
- `RequestInfoMessage` - Clasă de bază pentru tipurile de date ale cererilor
- `RequestResponse` - Corelează răspunsurile umane cu cererile
- `WorkflowOutputEvent` - Eveniment pentru detectarea rezultatelor fluxului de lucru


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Pasul 2: Definirea modelelor Pydantic pentru rezultate structurate

Aceste modele definesc **schema** pe care agenții o vor returna. Păstrăm toate modelele din fluxul de lucru condițional și adăugăm:

**Nou pentru Intervenția Umană:**
- `HumanFeedbackRequest` - Subclasă a `RequestInfoMessage` care definește datele de cerere trimise oamenilor
  - Conține `prompt` (întrebarea de adresat) și `destination` (context despre orașul indisponibil)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Pasul 3: Creează Instrumentul de Rezervare Hotelieră

Același instrument din fluxul de lucru condițional - verifică dacă există camere disponibile la destinație.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Pasul 4: Definirea funcțiilor de condiție pentru rutare

Avem nevoie de **patru funcții de condiție** pentru fluxul de lucru cu intervenție umană:

**Din fluxul de lucru condițional:**
1. `has_availability_condition` - Rutează când hotelurile SUNT disponibile
2. `no_availability_condition` - Rutează când hotelurile NU sunt disponibile

**Nou pentru intervenția umană:**
3. `user_wants_alternatives_condition` - Rutează când utilizatorul spune "da" la alternative
4. `user_declines_alternatives_condition` - Rutează când utilizatorul spune "nu" la alternative


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Pasul 5: Crearea Executorului Managerului de Decizii

Acesta este **nucleul modelului cu implicarea umană**! `DecisionManager` este un `Executor` personalizat care:

1. **Primește feedback uman** prin obiecte `RequestResponse`
2. **Procesează decizia utilizatorului** (da/nu)
3. **Direcționează fluxul de lucru** trimițând mesaje agenților corespunzători

Caracteristici cheie:
- Folosește decoratorul `@handler` pentru a expune metodele ca pași ai fluxului de lucru
- Primește `RequestResponse[HumanFeedbackRequest, str]` care conține atât cererea originală, cât și răspunsul utilizatorului
- Generează mesaje simple "da" sau "nu" care declanșează funcțiile noastre de condiție


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Pasul 6: Crearea unui Executor Personalizat pentru Afișare

Același executor de afișare din fluxul de lucru condițional - produce rezultatele finale ca ieșire a fluxului de lucru.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Pasul 7: Încarcă variabilele de mediu

Configurează clientul LLM (GitHub Models, Azure OpenAI sau OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Pasul 8: Crearea agenților AI și a executorilor

Creăm **șase componente ale fluxului de lucru**:

**Agenți (împachetați în AgentExecutor):**
1. **availability_agent** - Verifică disponibilitatea hotelului folosind instrumentul
2. **confirmation_agent** - 🆕 Pregătește cererea de confirmare umană
3. **alternative_agent** - Sugerează orașe alternative (când utilizatorul spune da)
4. **booking_agent** - Încurajează rezervarea (când camerele sunt disponibile)
5. **cancellation_agent** - 🆕 Gestionează mesajul de anulare (când utilizatorul spune nu)

**Executori speciali:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` care întrerupe fluxul de lucru pentru input uman
7. **decision_manager** - 🆕 Executor personalizat care direcționează pe baza răspunsului uman (definit deja mai sus)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Pasul 9: Construirea fluxului de lucru cu intervenția umană

Acum construim graficul fluxului de lucru cu **rutare condiționată**, incluzând calea cu intervenția umană:

**Structura fluxului de lucru:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Legături cheie:**
- `availability_agent → confirmation_agent` (când nu sunt camere disponibile)
- `confirmation_agent → prepare_human_request` (transformare tip)
- `prepare_human_request → request_info_executor` (pauză pentru intervenția umană)
- `request_info_executor → decision_manager` (întotdeauna - oferă RequestResponse)
- `decision_manager → alternative_agent` (când utilizatorul spune "da")
- `decision_manager → cancellation_agent` (când utilizatorul spune "nu")
- `availability_agent → booking_agent` (când camerele sunt disponibile)
- Toate căile se termină la `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Pasul 10: Rulează Testul 1 - Oraș FĂRĂ Disponibilitate (Paris cu Confirmare Umană)

Acest test demonstrează **ciclul complet cu intervenție umană**:

1. Solicită hotel în Paris
2. availability_agent verifică → Fără camere disponibile
3. confirmation_agent creează o întrebare pentru utilizator
4. request_info_executor **oprește fluxul de lucru** și emite `RequestInfoEvent`
5. **Aplicația detectează evenimentul și solicită utilizatorului în consolă**
6. Utilizatorul tastează "da" sau "nu"
7. Aplicația trimite răspunsul prin `send_responses_streaming()`
8. decision_manager direcționează în funcție de răspuns
9. Rezultatul final este afișat

**Model Cheie:**
- Folosește `workflow.run_stream()` pentru prima iterație
- Folosește `workflow.send_responses_streaming(pending_responses)` pentru iterațiile ulterioare
- Ascultă `RequestInfoEvent` pentru a detecta când este necesară intervenția umană
- Ascultă `WorkflowOutputEvent` pentru a captura rezultatele finale


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Pasul 11: Rulează Testul 2 - Oraș CU Disponibilitate (Stockholm - Fără Intervenție Umană Necesară)

Acest test demonstrează **calea directă** atunci când camerele sunt disponibile:

1. Solicită hotel în Stockholm
2. availability_agent verifică → Camere disponibile ✅
3. booking_agent sugerează rezervarea
4. display_result afișează confirmarea
5. **Nu este necesară intervenția umană!**

Fluxul ocolește complet calea cu intervenție umană atunci când camerele sunt disponibile.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Concluzii și cele mai bune practici pentru implicarea umană

### ✅ Ce ai învățat:

#### 1. **Modelul RequestInfoExecutor**
Modelul de implicare umană din Microsoft Agent Framework utilizează trei componente cheie:
- `RequestInfoExecutor` - Oprește fluxul de lucru și emite evenimente
- `RequestInfoMessage` - Clasă de bază pentru încărcături de cereri tipizate (extinde această clasă!)
- `RequestResponse` - Corelează răspunsurile umane cu cererile originale

**Înțelegere critică:**
- `RequestInfoExecutor` NU colectează inputul singur - doar oprește fluxul de lucru
- Codul aplicației tale trebuie să asculte evenimentul `RequestInfoEvent` și să colecteze inputul
- Trebuie să apelezi `send_responses_streaming()` cu un dicționar care mapează `request_id` la răspunsul utilizatorului

#### 2. **Modelul de execuție în flux**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Arhitectura bazată pe evenimente**
Ascultă evenimente specifice pentru a controla fluxul de lucru:
- `RequestInfoEvent` - Este necesar input uman (fluxul de lucru este oprit)
- `WorkflowOutputEvent` - Rezultatul final este disponibil (fluxul de lucru complet)
- `WorkflowStatusEvent` - Schimbări de stare (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, etc.)

#### 4. **Executori personalizați cu @handler**
`DecisionManager` demonstrează cum să creezi executori care:
- Utilizează decoratorul `@handler` pentru a expune metode ca pași ai fluxului de lucru
- Primesc mesaje tipizate (de exemplu, `RequestResponse[HumanFeedbackRequest, str]`)
- Direcționează fluxul de lucru prin trimiterea de mesaje către alți executori
- Accesează contextul prin `WorkflowContext`

#### 5. **Routare condițională cu decizii umane**
Poți crea funcții condiționale care evaluează răspunsurile umane:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Aplicații în lumea reală:

1. **Fluxuri de aprobare**
   - Obține aprobarea managerului înainte de procesarea rapoartelor de cheltuieli
   - Solicită revizuirea umană înainte de trimiterea e-mailurilor automate
   - Confirmă tranzacțiile de valoare mare înainte de execuție

2. **Moderarea conținutului**
   - Marchează conținutul discutabil pentru revizuirea umană
   - Cere moderatorilor să ia decizia finală în cazuri limită
   - Escaladează la oameni când încrederea AI este scăzută

3. **Serviciul pentru clienți**
   - Permite AI să gestioneze automat întrebările de rutină
   - Escaladează problemele complexe la agenți umani
   - Întreabă clientul dacă dorește să vorbească cu un om

4. **Procesarea datelor**
   - Cere oamenilor să rezolve intrările de date ambigue
   - Confirmă interpretările AI ale documentelor neclare
   - Permite utilizatorilor să aleagă între mai multe interpretări valide

5. **Sisteme critice pentru siguranță**
   - Solicită confirmarea umană înainte de acțiuni ireversibile
   - Obține aprobarea înainte de accesarea datelor sensibile
   - Confirmă deciziile în industrii reglementate (sănătate, finanțe)

6. **Agenți interactivi**
   - Construiește roboți conversaționali care pun întrebări suplimentare
   - Creează asistenți care ghidează utilizatorii prin procese complexe
   - Proiectează agenți care colaborează cu oamenii pas cu pas

### 🔄 Comparație: Cu vs Fără implicarea umană

| Caracteristică | Flux de lucru condițional | Flux de lucru cu implicare umană |
|----------------|---------------------------|----------------------------------|
| **Execuție** | Un singur `workflow.run()` | Buclă cu `run_stream()` + `send_responses_streaming()` |
| **Input utilizator** | Niciunul (complet automatizat) | Solicitări interactive prin `input()` sau UI |
| **Componente** | Agenți + Executori | + RequestInfoExecutor + DecisionManager |
| **Evenimente** | Doar AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, etc. |
| **Oprire** | Fără oprire | Fluxul de lucru se oprește la RequestInfoExecutor |
| **Control uman** | Fără control uman | Oamenii iau decizii cheie |
| **Caz de utilizare** | Automatizare | Colaborare și supraveghere |

### 🚀 Modele avansate:

#### Puncte multiple de decizie umană
Poți avea mai multe noduri `RequestInfoExecutor` în același flux de lucru:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Gestionarea timeout-urilor
Implementează timeout-uri pentru răspunsurile umane:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrare UI avansată
În loc de `input()`, integrează cu UI web, Slack, Teams, etc.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Implicare umană condițională
Solicită input uman doar în situații specifice:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Cele mai bune practici:

1. **Extinde întotdeauna RequestInfoMessage**
   - Oferă siguranță tipurilor și validare
   - Permite context bogat pentru redarea UI
   - Clarifică intenția fiecărui tip de cerere

2. **Folosește solicitări descriptive**
   - Include context despre ceea ce ceri
   - Explică consecințele fiecărei alegeri
   - Păstrează întrebările simple și clare

3. **Gestionează inputul neașteptat**
   - Validează răspunsurile utilizatorilor
   - Oferă valori implicite pentru inputul invalid
   - Oferă mesaje de eroare clare

4. **Urmărește ID-urile cererilor**
   - Utilizează corelarea între request_id și răspunsuri
   - Nu încerca să gestionezi manual starea

5. **Proiectează pentru non-blocare**
   - Nu bloca firele de execuție așteptând input
   - Utilizează modele asincrone pe tot parcursul
   - Suportă instanțe concurente ale fluxului de lucru

### 📚 Concepte conexe:

- **Middleware pentru agenți** - Interceptează apelurile agenților (notebook anterior)
- **Gestionarea stării fluxului de lucru** - Persistă starea fluxului de lucru între execuții
- **Colaborarea între agenți** - Combină implicarea umană cu echipele de agenți
- **Arhitecturi bazate pe evenimente** - Construiește sisteme reactive cu evenimente

---

### 🎓 Felicitări!

Ai stăpânit fluxurile de lucru cu implicare umană din Microsoft Agent Framework! Acum știi cum să:
- ✅ Oprești fluxurile de lucru pentru a colecta input uman
- ✅ Utilizezi RequestInfoExecutor și RequestInfoMessage
- ✅ Gestionezi execuția în flux cu evenimente
- ✅ Creezi executori personalizați cu @handler
- ✅ Direcționezi fluxurile de lucru pe baza deciziilor umane
- ✅ Construiești agenți AI interactivi care colaborează cu oamenii

**Acesta este un model critic pentru construirea sistemelor AI de încredere și controlabile!** 🚀



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
